In [1]:
# 下载文件
! wget https://github.com/LC1332/Needy-Haruhi/raw/main/data/Daily_event_130.jsonl

--2023-11-06 13:02:52--  https://github.com/LC1332/Needy-Haruhi/raw/main/data/Daily_event_130.jsonl
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LC1332/Needy-Haruhi/main/data/Daily_event_130.jsonl [following]
--2023-11-06 13:02:52--  https://raw.githubusercontent.com/LC1332/Needy-Haruhi/main/data/Daily_event_130.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106940 (104K) [text/plain]
Saving to: ‘Daily_event_130.jsonl’

Daily_event_130.jso 100%[===================>] 104.43K  --.-KB/s    in 0.02s   

2023-11-06 13:02:53 (4.33 MB/s) - ‘Daily_event_130.jsonl’ saved [106940/106940]



In [3]:
!pip install -q langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [6]:
# 设置openai key
openai.api_key = 'sk-xA6WE1hds4ewvsxxx'
openai.api_base = 'https://api.sirly.cc/v1'

In [15]:
# emoji方法
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, AIMessage
import openai



def generation_emoji_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    option_time = 0

    decoder = json.JSONDecoder()
    chat = ChatOpenAI(temperature=0, openai_api_key=openai.api_key, openai_api_base=openai.api_base)
    for i in range(len(lines)):
        objs = []
        s = lines[i]
        while s:
            obj, pos = decoder.raw_decode(s)
            objs.append(obj)
            s = s[pos:].lstrip()

        for obj in objs:
            prefix = obj['prefix']
            messages2 = [
                SystemMessage(
                    content="从现在开始，你是一个情感分析bot，我会给你一段角色糖糖的对话，你需要将该段对话转换为emoji,"
                            "注意：emoji的范围包括情感和代表事物的emoji。"),
                AIMessage(content="好的，我会严格遵守你的要求,把用户的对话转换为emoji。"),
                SystemMessage(content='''
                糖糖:阿P，看！我买了小发发'
                '''),
                AIMessage(content="😊"),
                SystemMessage(content='''
                糖糖:糖糖好像累了……让糖糖休息一下好不好'
                '''),
                AIMessage(content="😣😢"),
                AIMessage(content="😊"),
                SystemMessage(content='''
                糖糖:房间太乱了 想买个扫地机器人 可是房间这么乱 买来也动不了'
                '''),
                AIMessage(content="🏘😓"),
                SystemMessage(content=f'''
                A:{prefix}'
                ''')
            ]
            obj['source'] = 'Original_Generation'
            if 'prefix_emoji' not in obj or obj['prefix_emoji'] == '':
                obj['prefix_emoji'] = chat(messages2).content
                print('\nprefix_emoji: ', obj['prefix_emoji'])

            else:
                pass
            if len(obj['options']) <= 1:
                print('prefix: ', prefix)

            for option in obj['options']:
                option_time += 1
                # 生成emoji
                user = option['user']
                reply = option['reply']
                attribute_change = option['attribute_change']
                print('\nprefix：',obj['prefix'])
                print('prefix_emoji: ', obj['prefix_emoji'])

                messages = [
                    SystemMessage(
                        content="从现在开始，你是一个情感分析bot，我会给你一段糖糖和阿P的对话记录和糖糖的情感变化数值，你需要仔细分析对话记录和情感变化数值,"
                                "然后输出一个emoji来表示糖糖当前的情感状态。"
                                "注意：你需要尽可能的扩大emoji的范围，既包括情感emoji，也包括代表事物的emoji"),
                    AIMessage(content="好的，我会严格遵守你的要求。"),
                    SystemMessage(content='''
                    糖糖:阿P，看！我买了小发发'
                    "阿P:真好看，跟糖糖好像"
                    "糖糖:对吧！我不在的时候，你就把小花花当成糖糖，好好疼爱它吧！"
                    "糖糖的感情数值变化：Stress: -1
                    '''),
                    AIMessage(content="😊💖"),
                    SystemMessage(content='''
                    糖糖:今天有点想试试平时不会做的事'
                    "阿P:相爱"
                    "糖糖:……我们不是一直相爱的吗～？"
                    "糖糖的感情数值变化：Stress: -1
                    '''),
                    AIMessage(content="😓💦"),
                    SystemMessage(content='''
                    糖糖:睡太久了浑身无力～～……我可以就酱紫睡一辈子吗？'
                    "阿P:可以啊！"
                    "糖糖:好耶～～！！！"
                    "糖糖的感情数值变化：Stress: -1
                    '''),
                    AIMessage(content="😋🎉"),

                    SystemMessage(content=f'''
                    糖糖:{prefix}'
                    "阿p:{user}"
                    "糖糖:{reply}"
                    "糖糖的感情数值变化：{attribute_change}
                    '''
                                  )
                ]
                # 演示需要，已经注释判断语句了~
                # if 'option_emoji' not in option or option['option_emoji'] == '':
                option['option_emoji'] = chat(messages).content

                print('reply: ',option['reply'])
                print('option_emoji: ', option['option_emoji'])


            lines[i] = '\n'.join(json.dumps(obj, ensure_ascii=False) for obj in objs)

            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(''.join(lines))



In [14]:
# 生成emoji
generation_emoji_jsonl('/content/Daily_event_130.jsonl')


prefix： 好想把直播鸽了～～可是数据要输给别人了，得赶快加把劲～ 阿P帮帮我
prefix_emoji:  📹💻🏃‍♂️🆘
reply:  是嘛，看来你对我期望真高啊！我会加油的！
option_emoji:  😔💪

prefix： 好想把直播鸽了～～可是数据要输给别人了，得赶快加把劲～ 阿P帮帮我
prefix_emoji:  📹💻🏃‍♂️🆘
reply:  我不累，真的！不需要休息，求求你帮我准备一下，拜托！
option_emoji:  😫🙏

prefix： 好想把直播鸽了～～可是数据要输给别人了，得赶快加把劲～ 阿P帮帮我
prefix_emoji:  📹💻🏃‍♂️🆘
reply:  辛苦你了，阿P，还是你最好了！
option_emoji:  😊🙏

prefix： 完全想不出什么有趣的整活点子！我应该怎么办呢？
prefix_emoji:  🤔🤷‍♀️
reply:  哈哈哈，你的建议简直太可笑了！不过，也许可以试试，万一真的能笑出来呢？阿P是个小天才！
option_emoji:  😄🤔

prefix： 完全想不出什么有趣的整活点子！我应该怎么办呢？
prefix_emoji:  🤔🤷‍♀️
reply:  出去只会浪费时间，我的灵感已经被黑洞吞噬了，我根本没有创作的欲望
option_emoji:  😔💭

prefix： 完全想不出什么有趣的整活点子！我应该怎么办呢？
prefix_emoji:  🤔🤷‍♀️
reply:  对呀，我就喜欢你这种明智的建议！阿P是不是变聪明了？
option_emoji:  🤔🤗

prefix： 啊干脆把事情全鸽了去看番吧，去看番！！我最爱看动画片了！！！！！！！！！！！！！！！！！！！！！！！！！！！！
prefix_emoji:  📺🍿🥰
reply:  嗯嗯没关系，反正我一个人也能看，谢啦阿P
option_emoji:  😔📺

prefix： 啊干脆把事情全鸽了去看番吧，去看番！！我最爱看动画片了！！！！！！！！！！！！！！！！！！！！！！！！！！！！
prefix_emoji:  📺🍿🥰
reply:  诶～为什么啊，阿P你怎么这样，让人家一个人孤单地看动画片真的好可怜喔
option_emoji:  😢💔

prefix： 啊干脆把事情全鸽了去看番吧，去看番！！我最爱看动画片了！！！！

KeyboardInterrupt: ignored